# Multilayer Perceptron

The simplest form of the learning procedure is for layered networks,<br> which have a layer of input units at the bottom;<br> any number of intermediate layers;<br> and a layer of output units at the top.<br><br>
Connections within a layer or from higher to lower layers are forbidden, but the connections **can skip intermediate layers**.<br>
An input vector is presented to the network by setting the states of the input units.<br>
Then the states of the units in each layer are determined by applying equations (1) and (2) to the connections coming from the lower layers.<br>
All units within a layer have their states set in parallel, but different layers have their states set sequentially, starting at the bottom and working upwards until the states of the output units are determined.<br><br>

## Equation (1).
The total input $x_j$, to unit $j$ is a linear function of the outputs $y_i$, of the units that are connected to $j$ and of the weights $w_{ji}$, on these connections.
$$
x_j = \sum_i y_iw_{ji}
$$

In [1]:
def x_j(y, w):
    y_w_multi = []
    for y_i, w_ji in zip(y, w):
        y_w_multi.append(y_i * w_ji)
    x_j = sum(y_w_multi)
    return x_j

## Equation (2).
Units can be given biases by introducing an extra input to each unit which always has a value of 1.<br>
The weight on this extra input is called the bias and is equivalent to a threshold of the opposite sign.<br>
It can be treated just like the other weights.<br>
A unit has a real-valued output $y_j$, which is a non-linear function of its total input.
$$
y_j = \frac{1}{1+e^{-x_j}}
$$

In [5]:
import math

def sigmoid_activation(x):
    return 1/(1+math.exp**(-x))

It is not necessary to use exactly the functions given in equations (1) and (2).<br>
Any input-output function which has a bounded derivative will do.<br><br>
However, the use of a linear function for combining the inputs to a unit before applying the nonlinearity greatly simplifies the learning procedure.

# Equation (3)
The aim is to find a set of weights that ensure that for each input vector the output vector produced by the network is the same as (or sufficiently close to) the desired output vector.<br>
If there is a fixed, finite set of input-output cases, the total error in the performance of the network with a particular set of weights can be computed by comparing the actual and desired output vectors for every case.<br><br>
The total error $E$ is definded as:
$$
E = \frac{1}{2}\sum_c\sum_j(y_{j,c}-d_{j,c})^2
$$
$where$<br>
$c$ is an index over cases (input-output pairs),<br>
$j$ is an index over output units,<br>
$y$ is the actual state of an output unit,<br>
$d$ is its desired state


In [2]:
def calculate_total_error(Y, D):
    E = 0
    for y_c, d_c in zip(Y, D):
        yc = []
        for y_j, d_j in zip(y_c, d_j):
            yc.append((y_j - d_j)**2)
        E = E + sum(yc)
    E = 0.5 * E
    return E            

# Equation (4)

The backward pass starts by computing $\partial E / \partial y$ for each of the output units.<br>
Differentiating equation(3) for a particular case $c$, and suppressing the index $c$ gives.
$$
\partial E / \partial y_j = y_j - d_j
$$
We can then apply the chain rule to compute $\partial E / \partial x_j$
$$
\partial E / \partial x_j = \partial E / \partial y_j \cdot dy_j / dx_j
$$

In [3]:
def compute_gradient(Y, D, dy_dx):
    dE_dy = [y_j - d_j for y_j, d_j in zip(Y, D)]
    dE_dx = [dE_dy_j * dy_dx_j for dE_dy_j, dy_dx,j in zip(dE_dy, dy_dx)]
    return dE_dy, dE_dx

# Equation (5)
Differentiating equation(2) to get the value of $dy_j/dx_j$ and substituting gives:
$$
\partial E / \partial x_j = \partial E / \partial y_j \cdot y_j(1-y_j)
$$

In [6]:
def compute_dE_dx(Y, dE_dy):
    dE_dx = [dE_dy_j * y_j * (1 - y_j) for dE_dy_j, y_j in zip(dE_dy, Y)]
    return dE_dx

# Equation(6)
This means that we know how a change in the total input $x$ to an output unit will affect the error.<br>
But this total input is just a linear function of the states of the lower level units and it is also a linear function of the weights on the connections,<br>so it is easy to comput how the error will be affected by changing these states and weights.<br>
For a weight $w_{ji}$ from $i$ to $j$ the derivative is:
\begin{equation}
\begin{aligned}
    \partial E / \partial w_{ji} &= \partial E / \partial x_j \cdot \partial x_j / \partial w_{ji}\\
    &=\partial E / \partial x_j \cdot y_i
\end{aligned}
\end{equation}
and for the output of the $i^{th}$ unit the contribution to $\partial E / \partial y_i$ resulting from the effect of $i$ on $j$ is simply
$$
\partial E / \partial x_j \cdot \partial x_j / \partial y_i = \partial E / \partial x_j \cdot w_{ji}
$$

In [8]:
def compute_weight_and_output_gradients(dE_dx, Y, W):
    dE_dw = [[dE_dx_j * y_i for y_i in Y] for dE_dx_j in dE_dx]
    dE_dy = [sum(dE_dx_j * w_ji for dE_dx_j, w_ji in zip(dE_dx, w_row)) for w_row in W]

    return dE_dw, dE_dy

# Equation (7)
so taking into account all the connections emanating from unit $i$ we have
$$
\partial E / \partial y_i = \sum_j \partial E/\partial x_j \cdot w_{ji}
$$

In [9]:
def compute_gradient_dE_dy(dE_dx, W):
    dE_dy = [sum(dE_dx_j * w_row[i] for dE_dx_j, w_row in zip(dE_dx, W)) for i in range(len(W[0]))]

    return dE_dy


# Equation (8)
One way of using $\partial E / \partial w$ is to change the weights after every input-output case.<br>
This has the advantage that no separate memory is required for the derivatives.<br>
An alternative scheme, which we used in the research reported here, is to accumulate $\partial E / \partial w$ over all the input-output cases before changing the weights.<br>
The simplest version of gradient descent is to change each weight by an amount of proportional to the accumulated $\partial E / \partial w$.
$$
\Delta w = - \epsilon \partial E / \partial w
$$

In [ ]:
def update_weights(W, dE_dw, epsilon):
    updated_W = [[w_ji - epsilon * dE_dw_ji for w_ji, dE_dw_ji in zip(w_row, dE_dw_row)] 
                 for w_row, dE_dw_row in zip(W, dE_dw)]

    return updated_W

# Equation (9)
The equation(8) can be significantly improved, without sacrificing the simplicity and locality,<br> by using an acceleration method in which the current gradient is used to modify the velocity of the point in weight space instead of its position.
$$
\Delta w(t) = - \epsilon \partial E / \partial w(t) + \alpha \Delta w (t-1)
$$
$where$<br>$t$ is incremented by 1 for each sweep through the whole set of input-output cases,<br>
$\alpha$ is an exponential decay factor between 0 and 1 that determines the relative contribution of the current gradient and earlier gradients to the weight change.

In [10]:
def update_weights_with_momentum(W, dE_dw, prev_delta_W, epsilon, alpha):
    updated_W = []
    current_delta_W = []
    for w_row, dE_dw_row, prev_delta_W_row in zip(W, dE_dw, prev_delta_W):
        updated_w_row = []
        current_delta_w_row = []
        for w_ji, dE_dw_ji, prev_delta_w_ji in zip(w_row, dE_dw_row, prev_delta_W_row):
            delta_w_ji = -epsilon * dE_dw_ji + alpha * prev_delta_w_ji
            updated_w_row.append(w_ji + delta_w_ji)
            current_delta_w_row.append(delta_w_ji)
        updated_W.append(updated_w_row)
        current_delta_W.append(current_delta_w_row)

    return updated_W, current_delta_W

# Multilayer Neuron Network

In [12]:
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [67]:
class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.w1_2_3_4 = [[1, 10], [1, 10]]
        self.w5_6 = [[-40], [40]]
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward(self, X):
        self.z1_2 = np.dot(X, self.w1_2_3_4)
        self.h = self.sigmoid(self.z1_2)
        self.z3 = np.dot(self.h, self.w5_6)
        self.o = self.sigmoid(self.z3)
        return self.o

    def mse_loss(self, y_true, y_pred):
        return np.mean((y_true - y_pred)**2)

    def backward(self, X, y, y_pred, learning_rate):
        dc_do1 = -2 * (y - y_pred)
        do1_dz3 = y_pred * (1 - y_pred)
        dz3_dw5_6 = self.h
        dc_dw5_6  = dc_do1 * do1_dz3 * dz3_dw5_6
        self.w5_6 = self.w5_6 + learning_rate * -dc_dw5_6.T
        
        dc_dw1_2_3_4 = dc_do1 * do1_dz3 * np.dot(self.w5_6 * (self.h * (1 - self.h)).T, X)
        self.w1_2_3_4 = self.w1_2_3_4 + learning_rate * -dc_dw1_2_3_4.T
    
    def train(self, X_train, y_train, epochs, learning_rate):
        for epoch in tqdm(range(epochs)):
            for i in range(len(X_train)):
                y_pred = self.forward([X_train[i]])
                loss = self.mse_loss([y_train[i]], y_pred)
                self.backward([X_train[i]], [y_train[i]], y_pred, learning_rate)
            if np.mod(epoch, 100) == 0:
                print(f'epoch = {epoch}, loss = {loss}')
            
                    


In [68]:
X_train = np.random.randint(0, 2, (100, 2))
y_train = (X_train[:, 0] != X_train[:, 1]).astype(int)

In [69]:
X_train[1]

array([1, 0])

In [70]:
y_train[1]

1

In [60]:
mlp = MLP(input_size=2, hidden_size=50000, output_size=1)

In [71]:
mlp.train(X_train, y_train, epochs=5000, learning_rate=0.1)

  0%|          | 0/5000 [00:00<?, ?it/s]

epcoh = 0, loss = 0.00022015812030046683
epcoh = 100, loss = 0.00024805913810660646
epcoh = 200, loss = 0.0002476592420388722
epcoh = 300, loss = 0.0002472605859686342
epcoh = 400, loss = 0.00024686316246771246
epcoh = 500, loss = 0.00024646696432065315
epcoh = 600, loss = 0.0002460719845141572
epcoh = 700, loss = 0.0002456782162257503
epcoh = 800, loss = 0.0002452856528127462
epcoh = 900, loss = 0.0002448942878024028
epcoh = 1000, loss = 0.00024450411488210946
epcoh = 1100, loss = 0.00024411512789061324
epcoh = 1200, loss = 0.00024372732080899152
epcoh = 1300, loss = 0.00024334068775329397
epcoh = 1400, loss = 0.00024295522296626228
epcoh = 1500, loss = 0.0002425709208106972
epcoh = 1600, loss = 0.00024218777576241046
epcoh = 1700, loss = 0.0002418057824040047
epcoh = 1800, loss = 0.00024142493541886884
epcoh = 1900, loss = 0.00024104522958541942
epcoh = 2000, loss = 0.00024066665977198488
epcoh = 2100, loss = 0.0002402892209317278
epcoh = 2200, loss = 0.00023991290809774427
epcoh = 2

In [72]:
test_input = np.array([[0, 0]])
pred_output = mlp.forward(test_input)
print(f'predicted_output: {test_input}, {pred_output}')

test_input = np.array([[1, 0]])
pred_output = mlp.forward(test_input)
print(f'predicted_output: {test_input}, {pred_output}')

test_input = np.array([[0, 1]])
pred_output = mlp.forward(test_input)
print(f'predicted_output: {test_input}, {pred_output}')

test_input = np.array([[1, 1]])
pred_output = mlp.forward(test_input)
print(f'predicted_output: {test_input}, {pred_output}')

predicted_output: [[0 0]], [[0.00311704]]
predicted_output: [[1 0]], [[0.99008973]]
predicted_output: [[0 1]], [[0.98859112]]
predicted_output: [[1 1]], [[0.01510274]]
